In [2]:
import numpy as np
import pandas as pd
import datetime
import warnings
import gc

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, InputLayer, Add, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.utils import plot_model
from sklearn.metrics import log_loss

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
import math
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
import lightgbm
from lightgbm import LGBMClassifier, log_evaluation

from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [23]:
train_data = pd.read_feather('../input/parquet-files-amexdefault-prediction/train_data.ftr')
train_labels = pd.read_csv('/kaggle/input/amex-default-prediction/train_labels.csv').set_index('customer_ID')
test_data = pd.read_feather('../input/parquet-files-amexdefault-prediction/test_data.ftr')
target = train_labels.target.values
sub = pd.read_csv('/kaggle/input/amex-default-prediction/sample_submission.csv')

# **<font size=6 color="blue">Feature Engineering </font>**

# **Feature Aggregation** 
Create a new dataset based on aggregated information from EDA

In [24]:
# train dataset
trn_num_statements = train_data.groupby('customer_ID').size().sort_index()

train_agg_data = (train_data
                .groupby('customer_ID')
                .tail(1)
                .set_index('customer_ID', drop=True)
                .sort_index()
                .drop(['S_2'], axis='columns'))

train_agg_data['target'] = train_labels.target
train_agg_data['num_statements'] = trn_num_statements

train_agg_data.reset_index(inplace = True, drop = True)

In [25]:
# test dataset
tst_num_statements = test_data.groupby('customer_ID').size().sort_index()

test_agg_data = (test_data
                .groupby('customer_ID')
                .tail(1)
                .set_index('customer_ID', drop=True)
                .sort_index()
                .drop(['S_2'], axis='columns'))

# Merge the labels from the labels dataframe
test_agg_data['num_statements'] = tst_num_statements

test_agg_data.reset_index(inplace = True, drop = True)

# **Categorical Encoding** 
<font size=3 >One hot encoding </font>

In [26]:
# For train data
cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

trn_not_cat_features = [f for f in train_agg_data.columns if f not in cat_features]
tst_not_cat_features = [f for f in test_agg_data.columns if f not in cat_features]

In [27]:
encoder = OrdinalEncoder()
trn_encoded_features = encoder.fit_transform(train_agg_data[cat_features])

In [28]:
trn_encoded_features = pd.DataFrame(trn_encoded_features)
train_agg_data = pd.concat([train_agg_data[trn_not_cat_features], trn_encoded_features], axis = 1)

In [29]:
# For test data
tst_encoded_features = encoder.transform(test_agg_data[cat_features])
tst_encoded_features = pd.DataFrame(tst_encoded_features)

test_agg_data = pd.concat([test_agg_data[tst_not_cat_features], tst_encoded_features], axis = 1)
test_agg_data.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,1,2,3,4,5,6,7,8,9,10
0,0.568848,0.121399,0.010780,1.009766,0.006924,0.149414,0.000396,0.003576,0.10376,0.007397,...,1.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,NaN,5.0
1,0.841309,0.126465,0.016556,1.008789,0.009712,0.112183,0.006191,0.011383,NaN,NaN,...,1.0,1.0,0.0,3.0,0.0,1.0,1.0,0.0,NaN,5.0
2,0.697754,0.002724,0.001485,0.810059,0.002621,0.166138,0.004887,0.015945,NaN,0.105286,...,1.0,0.0,0.0,3.0,0.0,0.0,2.0,2.0,0.0,3.0
3,0.513184,0.324707,0.149536,0.205688,0.002277,0.181152,0.005814,0.498535,NaN,0.211670,...,2.0,0.0,0.0,4.0,0.0,1.0,0.0,1.0,NaN,4.0
4,0.254395,0.768066,0.563477,0.038025,0.502930,0.168335,0.009483,0.831055,NaN,0.071899,...,5.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,NaN,4.0


# **Imputation** 
<font size=3 > Impute missing values</font>

In [30]:
# fill missing values with a number (0)
train_agg_data.fillna(value = 0, inplace = True)
test_agg_data.fillna(value = 0, inplace = True)

# **Creating Features**

In [ ]:
features_avg = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_8', 'B_9', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14',\
                'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_28',\
                'B_29', 'B_30', 'B_32', 'B_33', 'B_37', 'B_38', 'B_39', 'B_40', 'B_41',  'D_39', 'D_41',\
                'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_50', 'D_51', 'D_53', 'D_54', 'D_55',\
                'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_65', 'D_66', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73',\
                'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_80', 'D_82', 'D_84', 'D_91', 'D_92', 'D_96',\
                'D_103', 'D_104', 'D_108', 'D_112', 'D_113', 'D_114', 'D_115', 'D_117', 'D_118', 'D_119', 'D_120',\
                'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126', 'D_128', 'D_129', 'D_131', 'D_132', 'D_133',\
                'D_134', 'D_135', 'D_136', 'D_140', 'D_141', 'D_142', 'D_144', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1',\
                'R_2', 'R_3', 'R_7', 'R_8', 'R_9', 'R_10', 'R_11', 'R_14', 'R_15', 'R_16', 'R_17', 'R_20', \
                'R_22',  'R_26', 'R_27', 'S_3', 'S_5', 'S_6', 'S_7', 'S_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16',\
                'S_18', 'S_22', 'S_23', 'S_25', 'S_26', 'B_42','D_86','D_94','R_21','R_24']
features_min = ['B_2', 'B_4', 'B_5', 'B_9', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_19', 'B_20', 'B_28',\
                'B_29', 'B_36',  'D_41', 'D_42', 'D_45', 'D_46', 'D_48', 'D_50', 'D_51',\
                'D_53', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_62', 'D_71', 'D_74', 'D_75', \
                'D_102', 'D_112', 'D_113', 'D_115', 'D_118', 'D_119', 'D_121', 'D_122', 'D_128', 'D_132',\
                'D_141', 'D_144', 'D_145', 'P_2', 'P_3', 'R_1', 'R_27', 'S_3', 'S_5', 'S_7', 'S_9', 'S_11',\
                'S_12', 'S_23', 'S_25','B_42','B_33', 'D_39', 'D_78','D_83','D_70','D_140']
features_max = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'B_10', 'B_12', 'B_13', 'B_14', 'B_15',\
                'B_16', 'B_17', 'B_18', 'B_19', 'B_21', 'B_23', 'B_24', 'B_25', 'B_29', 'B_30', 'B_37', 'B_38',\
                'B_40', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49',\
                'D_50', 'D_52', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_63', 'D_64', 'D_65', 'D_70', 'D_71',\
                'D_72', 'D_73', 'D_74', 'D_76', 'D_77', 'D_78', 'D_80', 'D_82', 'D_84', 'D_102', 'D_105', 'D_107',\
                'D_110', 'D_112', 'D_115', 'D_116', 'D_117', 'D_118', 'D_119', 'D_121', 'D_122', 'D_123', 'D_124',\
                'D_125', 'D_128', 'D_131', 'D_132', 'D_133', 'D_134', 'D_138',  'D_141',\
                'D_142', 'D_144', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_3', 'R_5', 'R_6', 'R_7','R_10', 'R_11', \
                'R_14',  'R_20', 'R_26', 'R_27', 'S_3', 'S_5', 'S_7', 'S_8', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', \
                'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27','B_42','B_39','D_91',  'R_8', 'D_136','D_140','D_111','D_126','D_135','B_33','R_17']
features_last = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', \
                 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_28',\
                 'B_29', 'B_30', 'B_33', 'B_36', 'B_37', 'B_38', 'B_39', 'B_40', 'D_39', 'D_41',\
                 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54',\
                 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_63', 'D_64', 'D_65', 'D_69', 'D_70', 'D_71',\
                 'D_72',  'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80',  'D_82', 'D_83', 'D_86', 'D_91',\
                 'D_96', 'D_105', 'D_106', 'D_112', 'D_114', 'D_119', 'D_120', 'D_121', 'D_122', 'D_124',  \
                 'D_127', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_138', 'D_140', 'D_141', 'D_142', \
                 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_2', 'R_3',  'R_5', 'R_6', 'R_7', 'R_9', 'R_10',\
                 'R_11', 'R_12', 'R_20', 'R_26', 'R_27', 'S_3', 'S_5', 'S_7', \
                 'S_8', 'S_9', 'S_11', 'S_12', 'S_13', 'S_16', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26',\
                 'S_27','B_32','B_41', 'B_42', 'D_73','D_81','D_125','D_126', 'R_4', 'R_14', 'R_8','R_13','R_15','R_19','S_6']


df_avg = (train_agg_data
          .groupby(cid)
          .mean()[features_avg]
          .rename(columns={f: f"{f}_avg" for f in features_avg})
         )
df_min = (train_agg_data
          .groupby(cid)
          .min()[features_min]
          .rename(columns={f: f"{f}_min" for f in features_min})
         )
df_max = (train_agg_data
          .groupby(cid)
          .max()[features_max]
          .rename(columns={f: f"{f}_max" for f in features_max})
         )

train_agg_data = (train_agg_data.loc[last, features_last]
      .rename(columns={f: f"{f}_last" for f in features_last})
      .set_index(np.asarray(cid[last]))
     )

train_agg_data = pd.concat([train_agg_data, df_min, df_max,df_avg], axis=1)

df_avg = (test_agg_data
          .groupby(cid)
          .mean()[features_avg]
          .rename(columns={f: f"{f}_avg" for f in features_avg})
         )
df_min = (test_agg_data
          .groupby(cid)
          .min()[features_min]
          .rename(columns={f: f"{f}_min" for f in features_min})
         )
df_max = (test_agg_data
          .groupby(cid)
          .max()[features_max]
          .rename(columns={f: f"{f}_max" for f in features_max})
         )

test_agg_data = (test_agg_data.loc[last, features_last]
      .rename(columns={f: f"{f}_last" for f in features_last})
      .set_index(np.asarray(cid[last]))
     )

test_agg_data = pd.concat([test_agg_data, df_min, df_max,df_avg], axis=1)

# **Scaling** 

In [ ]:
scaler = StandardScaler()

train_agg_data_ = scaler.fit_transform(train_agg_data)
test_data_scaled_ = scaler.transform(test_agg_data[features])

# **<font size=5.5 color="blue">Evaluation Metrics</font>**

In [ ]:
#amex metric
def amex_metric(y_true, y_pred, return_components=False) -> float:
    
    def top_four_percent_captured(df) -> float:
        
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum() 
    
    def weighted_gini(df) -> float:
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(df) -> float:
        
        df2 = pd.DataFrame({'target': df.target, 'prediction': df.target})
        df2.sort_values('prediction', ascending=False, inplace=True)
        return weighted_gini(df) / weighted_gini(df2)
    
    df = pd.DataFrame({'target': y_true.ravel(), 'prediction': y_pred.ravel()})
    df.sort_values('prediction', ascending=False, inplace=True)
    g = normalized_weighted_gini(df)
    d = top_four_percent_captured(df)

    if return_components: return g, d, 0.5 * (g + d)
    return 0.5 * (g + d)

# MAE, RMSE, R SQUARED, ACCURACY
def evaluate(y_test , y_pred):
    print("MAE: ",mean_absolute_error(y_test,y_pred) , '\n')
        
    print("RMSE: ",np.sqrt(mean_squared_error(y_test,y_pred)), '\n')
        
    r2 = r2_score(y_test,y_pred)
    print("R Squared: ",r2, '\n')
    
    accuracy = accuracy_score(y_test, y_pred, normalize=True)
    print("Accuracy: ",accuracy, '\n')

# **<font size=5.5 color="blue">Implementation & Validation</font>**

In [ ]:
# split dataset
y = train_agg_data['target']
X = train_agg_data.drop('target', axis=1)

x_train, x_test , y_train , y_test = train_test_split(X,y,test_size=0.3,random_state=42) 

# **KNN**

In [ ]:
# instantiate the model with k=5
knn_model = KNeighborsClassifier(n_neighbors=5)

# fit the model to the training set
knn_model.fit(x_train, y_train)

# predict on the test-set
y_pred_knn = knn_model.predict(x_test)

In [7]:
print('AMEX METRIC: ',amex_metric_np(y_test.values, y_pred_knn, return_components=False))
print('')
evaluate(y_test , y_pred_knn)


AMEX METRIC:  0.46709616342968685

MAE:  0.13537365307300916 

RMSE:  0.36793158749013266 

R Squared:  0.2945193916954576 

Accuracy:  0.8646263469269908 



In [ ]:
# make predictions
predictions_knn = knn_5.predict(test_dataset)
output_knn = pd.DataFrame({'customer_ID': sub.customer_ID, 'prediction': predictions_knn})
output_knn.to_csv('submission_knn.csv', index=False)

# **SVM**

In [ ]:
param_grid = { 
    'C': [6,7,8],
}
clf = LinearSVC(class_weight='balanced', dual=False, penalty = "l1", random_state = 42)
svc_random = GridSearchCV(estimator = clf, param_grid = param_grid, cv=2, verbose=0, n_jobs = -1)
svc_random.fit(x_train,y_train)
svc_model = svc_random.best_estimator_

y_pred_svm = svc_model.predict(x_test)

In [8]:
print('AMEX METRIC: ',amex_metric(y_test, y_pred, return_components=False))
print('')
evaluate(y_test , y_pred)


AMEX METRIC:  0.5233161525009339

MAE:  0.11808286937668196 

RMSE:  0.343631880617445 

R Squared:  0.3846278605388628 

Accuracy:  0.881917130623318



In [ ]:
# make predictions
predictions = svc_model.predict(test_dataset[num_columns])
output_svm = pd.DataFrame({'customer_ID': sub.customer_ID, 'prediction': predictions})
output_svm.to_csv('submission_svm.csv', index=False)

# **LGBMClassifier**

In [2]:
features = [f for f in train_agg_data.columns if f != 'customer_ID' and f != 'target' ]

def my_booster(random_state=1, n_estimators=1300):
    return LGBMClassifier(n_estimators=n_estimators,
                          learning_rate=0.03, reg_lambda=50,
                          min_child_samples=2400,
                          num_leaves=94,
                          colsample_bytree=0.19,
                          max_bins=517, random_state=random_state)
      
score_list = []
y_pred_list = []
kf = StratifiedKFold(n_splits=5)
for fold, (idx_tr, idx_va) in enumerate(kf.split(train_agg_data, target)):
    X_tr, X_va, y_tr, y_va, model = None, None, None, None, None
    start_time = datetime.datetime.now()
    X_tr = train_agg_data.iloc[idx_tr][features]
    X_va = train_agg_data.iloc[idx_va][features]
    y_tr = target[idx_tr]
    y_va = target[idx_va]
    
    model = my_booster()
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=UserWarning)
        model.fit(X_tr, y_tr,
                  eval_set = [(X_va, y_va)], 
                  eval_metric=[lgb_amex_metric],
                  callbacks=[log_evaluation(100)])
    X_tr, y_tr = None, None
    y_va_pred = model.predict_proba(X_va, raw_score=True)
    score = amex_metric(y_va, y_va_pred)
    n_trees = model.best_iteration_
    if n_trees is None: n_trees = model.n_estimators
    print(f"Fold {fold} | {str(datetime.datetime.now() - start_time)[-12:-7]} |"
          f" {n_trees:5} trees |"
          f"                Score = {score:.5f}")
    score_list.append(score)
    
    if INFERENCE:
        y_pred_list.append(model.predict_proba(test_agg_data[features], raw_score=True))
            
print(f"OOF Score:                       {np.mean(score_list):.5f}")


[100]	valid_0's binary_logloss: 0.246906	valid_0's amex: 0.764519
[200]	valid_0's binary_logloss: 0.228331	valid_0's amex: 0.780338
[300]	valid_0's binary_logloss: 0.223375	valid_0's amex: 0.785591
[400]	valid_0's binary_logloss: 0.221115	valid_0's amex: 0.78835
[500]	valid_0's binary_logloss: 0.219791	valid_0's amex: 0.790354
[600]	valid_0's binary_logloss: 0.218981	valid_0's amex: 0.791777
[700]	valid_0's binary_logloss: 0.218384	valid_0's amex: 0.792521
[800]	valid_0's binary_logloss: 0.217952	valid_0's amex: 0.793035
[900]	valid_0's binary_logloss: 0.217702	valid_0's amex: 0.793741
[1000]	valid_0's binary_logloss: 0.217524	valid_0's amex: 0.793917
[1100]	valid_0's binary_logloss: 0.217322	valid_0's amex: 0.793403
[1200]	valid_0's binary_logloss: 0.217244	valid_0's amex: 0.793542
[1300]	valid_0's binary_logloss: 0.21716	valid_0's amex: 0.794232
[1400]	valid_0's binary_logloss: 0.217156	valid_0's amex: 0.794385
[1500]	valid_0's binary_logloss: 0.217179	valid_0's amex: 0.794007
Fold 

In [ ]:
sub_lgb = pd.DataFrame({'customer_ID': test.index,'prediction': np.mean(y_pred_list, axis=0)})
sub_lgb.to_csv('submission_lgb.csv', index=False)

# **Neural Network**

In [31]:
# feature selection
features = [f for f in train_agg_data.columns if f != 'target' and f != 'customer_ID']

In [32]:
import gc
del train_data, test_data
gc.collect()

99

In [35]:
# NN model

def nn_model():
    regularization = 4e-4
    activation_func = 'swish'
    inputs = Input(shape = (len(features)))

    x0 = Dense(256,kernel_regularizer = tf.keras.regularizers.l2(regularization), activation = activation_func)(inputs)
    x1 = Dense(128,kernel_regularizer = tf.keras.regularizers.l2(regularization),activation = activation_func)(x0)
    x1 = Dense(64,kernel_regularizer = tf.keras.regularizers.l2(regularization),activation = activation_func)(x1)
    x1 = Dense(32,kernel_regularizer = tf.keras.regularizers.l2(regularization),activation = activation_func)(x1)
    
    x1 = Concatenate()([x1, x0])
    x1 = Dropout(0.1)(x1)
    
    x1 = Dense(16, kernel_regularizer=tf.keras.regularizers.l2(regularization),activation=activation_func,)(x1)
    
    x1 = Dense(1, activation='sigmoid')(x1)
    
    model = Model(inputs, x1)
    
    return model

In [36]:
#model parameters
BATCH_SIZE         = 2048
EPOCHS             = 192 
EPOCHS_COSINEDECAY = 192 
DIAGRAMS           = True
USE_PLATEAU        = False
INFERENCE          = False
VERBOSE            = 0 
TARGET             = 'target'

In [37]:
# Train model
def fit_model(X_train, y_train, X_val, y_val, run = 0):

    start = 0.01
    start_time = datetime.datetime.now()
    
    X_train = scaler.fit_transform(X_train)

    epochs = EPOCHS    
    lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.7, patience = 4, verbose = VERBOSE)
    es = EarlyStopping(monitor = 'val_loss',patience = 12, verbose = 1, mode = 'min', restore_best_weights = True)
    tm = tf.keras.callbacks.TerminateOnNaN()
    callbacks = [lr, es, tm]
    
    if USE_PLATEAU == False:
        epochs = EPOCHS_COSINEDECAY
        end = 0.0002

        def cosine_decay(epoch):
            if epochs > 1:
                w = (1 + math.cos(epoch / (epochs - 1) * math.pi)) / 2
            else:
                w = 1
            return w * start + (1 - w) * end
        
        lr = LearningRateScheduler(cosine_decay, verbose = 0)
        callbacks = [lr, tm]

    model = nn_model()
    optimizer_func = tf.keras.optimizers.Adam(learning_rate = start)
    loss_func = tf.keras.losses.BinaryCrossentropy()
    model.compile(optimizer = optimizer_func, loss = loss_func)
    
    X_val = scaler.transform(X_val)
    validation_data = (X_val, y_val)
    
    history = model.fit(X_train, 
                        y_train, 
                        validation_data = validation_data, 
                        epochs          = epochs,
                        verbose         = VERBOSE,
                        batch_size      = BATCH_SIZE,
                        shuffle         = True,
                        callbacks       = callbacks
                       )
    
    history_list.append(history.history)
    
    print(f'Training Loss: {history_list[-1]["loss"][-1]:.5f}, Validation Loss: {history_list[-1]["val_loss"][-1]:.5f}')
    callbacks, es, lr, tm, history = None, None, None, None, None
    
    y_val_pred = model.predict(X_val, batch_size = BATCH_SIZE, verbose = VERBOSE).ravel()
    amex_score = amex_metric(y_val.values, y_val_pred, return_components = False)
    
    print(f'Fold {run}.{fold} '
          f' Amex Score: {amex_score:.5f}')
    
    print('')
    
    score_list.append(amex_score)
    
    test_data_scaled = scaler.transform(test_agg_data[features])
    tst_pred = model.predict(test_data_scaled)
    predictions.append(tst_pred)
    
    return model

In [1]:
history_list = []
score_list   = []
predictions  = []

kf = StratifiedKFold(n_splits = 5)

for fold, (trn_idx, val_idx) in enumerate(kf.split(train_agg_data, target)):
    X_train, X_val = train_agg_data.iloc[trn_idx][features], train_agg_data.iloc[val_idx][features]
    y_train, y_val = train_agg_data.iloc[trn_idx][TARGET], train_agg_data.iloc[val_idx][TARGET]
    
    fit_model(X_train, y_train, X_val, y_val)
    
print(f'OOF AUC: {np.mean(score_list):.5f}')


Training Loss: 0.22565, Validation Loss: 0.22929
Fold 0.0 Amex Score: 0.78218

Training Loss: 0.22602, Validation Loss: 0.22927
Fold 0.1 Amex Score: 0.78031

Training Loss: 0.22648, Validation Loss: 0.22678
Fold 0.2 Amex Score: 0.78267

Training Loss: 0.22654, Validation Loss: 0.22634
Fold 0.3 Amex Score: 0.78491

Training Loss: 0.22649, Validation Loss: 0.22593
Fold 0.4 Amex Score: 0.78619

OOF AUC: 0.78325

